# Rag From Scratch: Query Transformations

Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.

![Screenshot 2024-03-25 at 8.08.30 PM.png](https://i.imgur.com/jwRw5Je.png)

## Enviornment

`(1) Packages`

In [1]:
%pip install langchain_community==0.3.3 tiktoken==0.8.0 langchain-openai==0.2.3 langchainhub==0.1.21 chromadb==0.5.15 langchain==0.3.4

In [2]:
# %pip install -U --quiet langchain-nvidia-ai-endpoints==0.3.1
%pip install langchain-together==0.2.0

  Using cached langchain_together-0.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain_together-0.2.0-py3-none-any.whl (12 kB)


`(2) LangSmith`

https://docs.smith.langchain.com/

In [ ]:
from google.colab import userdata
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

`(3) API Keys`

In [5]:
# nvidia_api_key = userdata.get('NVIDIA_API_KEY')
# if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
#     os.environ["NVIDIA_API_KEY"] = nvidia_api_key
from google.colab import userdata
import os

os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')

## Part 5: Multi Query

Flow:

![Screenshot 2024-02-12 at 12.39.59 PM.png](https://i.imgur.com/kXGaadn.png)

Docs:

* https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever

### Index

In [6]:
# from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# embedder=NVIDIAEmbeddings(model="NV-Embed-QA")
# llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-70b-instruct") #You should change your model name to be matched with NVIDIA_API_KEY
# result = llm.invoke("Hãy giới thiệu về RAG pipeline cho mọi người.")
# print(result.content)

from langchain_together import ChatTogether, TogetherEmbeddings

llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf"
)
embd = TogetherEmbeddings(model="BAAI/bge-large-en-v1.5")
result = llm.invoke("Let's introduce RAG pipeline in three sentences.")
print(result.content)

Here is a three-sentence introduction to the RAG (Retrieval-Augmented Genome) pipeline:

The RAG pipeline is a novel approach to natural language processing (NLP) that combines the powers of retrieval and generation to produce more accurate and informative text outputs. RAG works by first retrieving relevant texts from a massive database and then using a generative model to produce a output that is conditioned on the retrieved texts, allowing it to generate more accurate and context-specific responses. By integrating retrieval and generation capabilities, RAG has the potential to significantly improve the performance of NLP models on a wide range of tasks, from question answering to text summarization and beyond.


In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=embd)

retriever = vectorstore.as_retriever()

### Prompt

In [8]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [9]:
import re

def is_match_question(question):
  """check whether question is match with `order_number. question` pattern"""
  pattern = r'^\d+\.\s+'
  match = re.match(pattern, question)
  return match

def filter_valid_question(questions):
  """return a list of valid questions which has format `order_number. question content`"""
  valid_questions = [question for question in questions
                     if question != "" and is_match_question(question)]
  return valid_questions

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}
Note: Only return a list of questions under format:
1. question 1
2. question 2
...
without any explaination"""



prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_perspectives
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
    | filter_valid_question
)

In [11]:
generate_queries.invoke("What is RAG pipeline?")

['1. What is the RAG processing pipeline?',
 '2. Can you explain the RAG data processing pipeline?',
 '3. What are the components of the RAG pipeline architecture?',
 '4. How does the RAG pipeline work in natural language processing?',
 '5. What is the purpose of the RAG pipeline in text generation models?']

In [15]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]): # list[list_documents_of_question1, list_documents_of_question2, ...,list_documents_of_question5]
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
# retrieval_chain = generate_queries
docs = retrieval_chain.invoke({"question":question})
len(docs)

9

In [16]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Finite context length: The restricted context capacity limits the inclusion of historical information, detailed instructions, API call context, and responses. The design of the system has to work with this limited communication bandwidth, while mechanisms like self-reflection to learn from past mistakes would benefit a lot from long or infinite context windows. Although vector stores and retrieval can provide access to a larger knowledge pool, their representation power is not as powerful as full attention.\n\n\nChallenges in long-term planning and task decomposition: Planning over a lengthy history and effectively exploring the solution space remain challenging. LLMs struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error.\n\n\nReliability of natural language interface: Current agent system relies on natural 

In [17]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'According to the context, task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This can be done in several ways, including:\n\n1. Using simple prompting techniques, such as "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, such as "Write a story outline." for writing a novel\n3. With human inputs\n\nAdditionally, techniques like Chain of Thought (CoT) and Tree of Thoughts can be used to decompose tasks into smaller steps and explore multiple reasoning possibilities at each step.'

## Part 6: RAG-Fusion

Flow:

![Screenshot 2024-02-12 at 12.41.36 PM.png](https://lh3.googleusercontent.com/pw/AP1GczPt7PX9JUTxSxXEeqWgMglvK4zIe5FeS9_wrCZSQ3O0qesW1OyIIFPQ3v14OceldfnuLJ3M3zJleyVXb1SLaA_YDO_-1XPIcg-698IkdrBq6N3uxAhacQK4otgiiBPs5Qv3OU1UHCzKV86Xxs5GMURP=w1824-h538-s-no-gm?authuser=0)

Docs:

* https://github.com/langchain-ai/langchain/blob/master/cookbook/rag_fusion.ipynb?ref=blog.langchain.dev

Blog / repo:

* https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1

### Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):
Note: Only return a list of questions under format:
1. question 1
2. question 2
...
without any explaination"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
import re

def is_match_question(question):
  """check whether question is match with `order_number. question` pattern"""
  pattern = r'^\d+\.\s+'
  match = re.match(pattern, question)
  return match

def filter_valid_question(questions):
  """return a list of valid questions which has format `order_number. question content`"""
  valid_questions = [question for question in questions
                     if question != "" and is_match_question(question)]
  return valid_questions

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
    | filter_valid_question
)

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

In [ ]:
# Documents: A, B, C, D
# - Question 1 --> Docs 1: [A, B, C, D]
# - Question 2 --> Docs 2: [A, D, B, C]
# - Question 3 --> Docs 3: [A, B, C]
# k=2
# Tính điểm cho A:
 # += 1/(1+k)=1/(1+2)=1/3
 # score_A = 1/3+1/3+1/3 = 1
# Tính điểm cho B:
 # Docs 1: 1/(2+k) = 1/(2+2) = 1/4
 # Docs 2: 1/(3+k) = 1/(3+2) = 1/5
 # Docs 3: 1/(2+k) = 1/(2+2) = 1/4
 # score_B = 1/4+ 1/5 + 1/4 =7/10
# Tính điểm cho C:
 # score_C = 1/5+1/6+1/5 = 17/30
# Tính điểm cho D:
 # score_D = 1/6 + 1/3 = 1/2

In [ ]:
docs

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

question = "What is task decomposition for LLM agents?"
final_rag_chain.invoke({"question":question})

Trace:

https://smith.langchain.com/public/071202c9-9f4d-41b1-bf9d-86b7c5a7525b/r

## Part 7: Decomposition

In [18]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):
Note: Only return a list of questions under format:
1. question 1
2. question 2
...
without any explaination"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [19]:
import re

def is_match_question(question):
  """check whether question is match with `order_number. question` pattern"""
  pattern = r'^\d+\.\s+'
  match = re.match(pattern, question)
  return match

def filter_valid_question(questions):
  """return a list of valid questions which has format `order_number. question content`"""
  valid_questions = [question for question in questions
                     if question != "" and is_match_question(question)]
  return valid_questions

In [20]:
# from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_core.output_parsers import StrOutputParser

# embedder=NVIDIAEmbeddings(model="NV-Embed-QA")
# llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-70b-instruct") #You should change your model name to be matched with NVIDIA_API_KEY

from langchain_together import ChatTogether, TogetherEmbeddings

llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf"
)
embd = TogetherEmbeddings(model="BAAI/bge-large-en-v1.5")

# Chain
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
    | filter_valid_question
)

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [21]:
questions

['1. What are the key functionalities of a Large Language Model (LLM) in an autonomous agent system?',
 '2. How does an autonomous agent system integrate natural language processing (NLP) with other AI components?',
 '3. What are the primary interfaces and APIs used to connect an LLM to other components in an autonomous agent system?']

### Answer recursively  

![Screenshot 2024-02-18 at 1.55.32 PM.png](https://i.imgur.com/PllWgtH.png)

Papers:

* https://arxiv.org/pdf/2205.10625.pdf
* https://arxiv.org/abs/2212.10509.pdf

In [23]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [24]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm

# llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-70b-instruct") #You should change your model name to be matched with NVIDIA_API_KEY


q_a_pairs = ""
for q in questions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [25]:
answer

'Based on the provided context and background question + answer pairs, the primary interfaces and APIs used to connect an LLM to other components in an autonomous agent system are:\n\n1. **Planning Domain Definition Language (PDDL)**: As mentioned in the context, LLM+P (Liu et al. 2023) uses PDDL as an intermediate interface to describe the planning problem. This allows the LLM to translate the problem into "Problem PDDL", request a classical planner to generate a PDDL plan, and then translate the PDDL plan back into natural language.\n\n2. **Memory Component Interface**: The LLM interacts with the agent\'s memory component to store and retrieve information, enabling the LLM to learn from past actions and refine its decision-making process.\n\n3. **External Tool APIs**: As seen in the ReAct (Yao et al. 2023) example, the LLM can interact with external tools, such as the Wikipedia search API, to gather information and perform tasks.\n\n4. **Neural-Symbolic Architecture APIs**: In the MR

Trace:

Question 1: https://smith.langchain.com/public/faefde73-0ecb-4328-8fee-a237904115c0/r

Question 2: https://smith.langchain.com/public/6142cad3-b314-454e-b2c9-15146cfcce78/r

Question 3: https://smith.langchain.com/public/84bdca0f-0fa4-46d4-9f89-a7f25bd857fe/r

### Answer individually

![Screenshot 2024-02-18 at 2.00.59 PM.png](https://i.imgur.com/uR6Qorn.png)

In [26]:
# Answer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
        rag_results.append(answer)

    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<ipython-input-26-37e16c633e1f>:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [27]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'Based on the provided Q+A pairs, the main components of an LLM-powered autonomous agent system can be synthesized as follows:\n\n1. **Large Language Model (LLM)**: Serves as the "brain" of the system, responsible for planning, reflection, and refinement. It breaks down large tasks into smaller manageable subgoals, learns from mistakes, and interacts with other components through a natural language interface.\n2. **Memory**: A key component that complements the LLM, although its exact role is not explicitly defined.\n3. **Planning**: A primary decision-making component that involves subgoal decomposition and reflection.\n4. **Expert Modules** (e.g., in a neuro-symbolic architecture like MRKL): These modules can be neural or symbolic and are used to process specific tasks or inquiries routed by the LLM.\n5. **External Tools and APIs**: The system can utilize these tools and APIs, with the LLM determining when to use them based on its capability.\n6. **Sensors**: Although not explicitly 

Trace:

https://smith.langchain.com/public/d8f26f75-3fb8-498a-a3a2-6532aa77f56b/r

## Part 8: Step Back

![Screenshot 2024-02-12 at 1.14.43 PM.png](https://i.imgur.com/S1ctQLf.png)

Paper:

* https://arxiv.org/pdf/2310.06117.pdf

In [28]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [29]:
# llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-70b-instruct") #You should change your model name to be matched with NVIDIA_API_KEY

In [30]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'how do AI agents break down complex tasks?'

In [31]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM agents refers to the process of breaking down a complex task into smaller, manageable subtasks or subgoals. This is an essential component of planning in a Large Language Model (LLM) powered autonomous agent system. The agent needs to decompose a task into smaller steps to plan ahead and efficiently handle complex tasks.\n\nThere are several ways to perform task decomposition, including:\n\n1. Using simple prompting techniques, such as "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, such as "Write a story outline." for writing a novel\n3. With human inputs\n\nAdditionally, techniques like Chain of Thought (CoT) and Tree of Thoughts can be used to enhance task decomposition. CoT involves instructing the model to "think step by step" to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at ea

## Part 9: HyDE

![Screenshot 2024-02-12 at 1.12.45 PM.png](https://i.imgur.com/BCIcjyS.png)

Docs:

* https://github.com/langchain-ai/langchain/blob/master/cookbook/hypothetical_document_embeddings.ipynb

Paper:

* https://arxiv.org/abs/2212.10496

In [32]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage, which is less than 200 tokens, to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

# from langchain_core.output_parsers import StrOutputParser
# from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-70b-instruct") #You should change your model name to be matched with NVIDIA_API_KEY


generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Here is a scientific paper passage answering the question:\n\nTask decomposition is a crucial strategy employed by Large Language Models (LLMs) to tackle complex tasks by breaking them down into a series of simpler, manageable sub-tasks. This decomposition enables LLMs to exploit their strengths in language understanding and generation while mitigating the risks associated with tackling intricate tasks as a single, monolithic entity. By recursively decomposing tasks into smaller sub-tasks, LLMs can iteratively refine their understanding of the task requirements, allocate resources more efficiently, and ultimately produce more accurate and effective solutions. This approach has been shown to improve the performance and robustness of LLMs in a wide range of natural language processing tasks, including text classification, question answering, and dialogue generation.'

In [33]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})